In [70]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mplt
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [71]:
pip install xgboost

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [72]:
# check xgboost version
import xgboost
print(xgboost.__version__)

1.5.0


In [73]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
submission_file = pd.read_csv('data/sample_submission.csv')

In [74]:
train_data.drop(columns=['id'], axis=1, inplace=True)

In [75]:
print(train_data.shape)
print(test_data.shape)

(600000, 101)
(540000, 101)


In [76]:
X, y = train_data.drop(columns = ['target']), train_data['target']

In [77]:
dtypes = train_data.dtypes
dtypes = dtypes[dtypes != 'object']
features = list(set(dtypes.index) - set(['target']))

len(features)

100

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [80]:
scaler = StandardScaler()
num_cols = X_train.select_dtypes(['integer', 'float']).columns

scaler.fit(X_train[num_cols])
X_train = pd.DataFrame(scaler.transform(X_train[num_cols]), columns = num_cols)
X_test = pd.DataFrame(scaler.transform(X_test[num_cols]), columns = num_cols)


In [81]:
# create an xgboost regression model
model = xgboost.XGBRegressor()
#model = xgboost.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)


In [82]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [83]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

In [84]:
auc_score_train = roc_auc_score(y_train, y_hat_train) * 100
auc_score_test = roc_auc_score(y_test, y_hat_test) * 100

print("ROC_AUC Score = {:.2f}% of XGBRegressor model on the training data.".format(auc_score_train))
print("ROC_AUC Score = {:.2f}% of XGBRegressor model on the validation data.".format(auc_score_test))

ROC_AUC Score = 78.49% of XGBRegressor model on the training data.
ROC_AUC Score = 73.02% of XGBRegressor model on the validation data.


In [85]:
def generate_results(model, df_test_data, features_column, id_column, target, file):

    df_test = df_test_data[features]
    results = df_test_data[[id_column]]
    results[target] = model.predict(df_test)
    results.to_csv(file, index=False, columns=results.columns)



In [86]:
generate_results(model, test_data, features, 'id', 'target', 'results/results.csv')

<ipython-input-85-60ceae91e977>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[target] = model.predict(df_test)
